# Multi Class Ship Classification by Transfer Learning (XCeption)

Applications of autonomous ships and underwater vessels are increasing in the maritime industry.

Some of the applications I can think of for autonomous vessels can be listed as:

* Maritime logistics, 
* Military and rescue operations, 
* Guarding and protecting important bays/marine protected areas/offshore platforms, 
* Oceanographic/hydrograpic surveys, 
* Marine biology research activities,
* Fish farm observations. 

Autonomous vessels need to identify nearby objects in order to conduct these operations safely. There are several sensors and components like 360 cameras, RADAR, automatic identification system (AIS), GPS, echo sounders (SONARs), which can be used onboard autonomous ships to increase situational awareness.

In this study, I want to focus on image data. From the camera onboard autonomous ships, we can detect and identify nearby objects, especially ships. With the help of computer vision and deep learning algorithms, we can classify different ship types.

Individual ship types can be compared from the GPS and AIS data to verify that individual ship's identity. Autonomous ships can detect and report illegal fishing or migration boats to the national authorities.

The dataset used in this study is from Analytics Vidhya. 6252 ship images of 5 classes are divided into train, validation and test splits. Data augmentation is applied in order to increase the model performance. Instead of creating a custom convolutional neural network (CNN), a pre-trained Xception model is used as the base model.

Model is evaluated with the confusion matrix, accuracy, precision, recall and F1 scores.

At the end of the study, 5 random images from another dataset are used for an additional demonstration of the model performance.

Resources

Dataset: https://www.kaggle.com/arpitjain007/game-of-deep-learning-ship-datasets

Photo: https://www.liquid-robotics.com/

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import os
from IPython.display import Image

print('Example of an autonomous vessel: Liquid Robotics Wave Glider')
display(Image("../input/autonomous-ship-images/liquid_robotics.jpeg", width=768, height=384))

## Importing the Dataset

There are 8932 images in the dataset. 6252 of the images are separated for training and only those images are labeled. 

6252 labeled images are used in the study and they will be splitted as train, validation and test data.

In the first part, csv file containing the image names and corresponding ship categories is imported.

In [ ]:
data_csv = pd.read_csv('../input/game-of-deep-learning-ship-datasets/train/train.csv')
print('Data shape: ', data_csv.shape)

categories = {0: 'Cargo', 1:'Military', 2:'Carrier', 3:'Cruise', 4:'Tankers'}

# I want classes to start from 0. So I subtracted 1 from the categories
data_csv['category'] = data_csv['category'] - 1
data_csv['label'] = data_csv['category'].map(categories)
data_csv['label']  = pd.Categorical(data_csv['label'])
data_csv.head(5)

As one can see from the graph below, the dataset is not equally distributed between the categories. Because this is not an academic study or part of an industry project, I am not going to manipulate the dataset. I will use all the images.

In [ ]:
sns.countplot(data_csv['label'])
plt.title('Ship Category Distribution')
plt.xlabel('Categories')
plt.ylabel('Counts of Observations');

In order to import the images, first of all file path is defined. 

Then each image name from the csv file is extracted in a loop and corresponding image is read. 

Later, each image is converted into RGB format and resized to have the same dimensions.

Lastly, each image is saved to a list in Numpy array format.

In [ ]:
# Importing the images.

import cv2

path = '../input/game-of-deep-learning-ship-datasets/train/images/'

# List of image names
img_list = list(data_csv['image'])

data_img = []

for each in img_list:
    # Each image path
    each_path = os.path.join(path, each)
    # Read each image
    each_img = cv2.imread(each_path)
    # OpenCv default color is BGR. Convert it to RGB
    each_img = cv2.cvtColor(each_img, cv2.COLOR_BGR2RGB)
    # Resize the images
    each_img_resized = cv2.resize(each_img, (128,128))
    # Save arrays to a list
    data_img.append(each_img_resized)

# Converting list to numpy array
X = np.array(data_img)
print('Shape of X: ', X.shape)

Labels from the csv file is extracted and one hot encoding is applied to have a target feature (dependent variable).

In [ ]:
from sklearn.preprocessing import OneHotEncoder

y = OneHotEncoder(dtype='int8', sparse=False).fit_transform(data_csv['category'].values.reshape(-1,1))
print('Shape of y: ', y.shape)

Let's look at some random images and their labels.

In [ ]:
indices = np.random.randint(0,6252,8) 
i = 1
plt.figure(figsize=(14,7))
for each in indices:
    plt.subplot(2,4,i)
    plt.imshow(X[each])
    plt.title(data_csv['label'].loc[each])
    plt.xticks([])
    plt.yticks([])
    i += 1

## Train, Validation and Test Split

The dataset is divided into train, validation and test splits.

In [ ]:
from sklearn.model_selection import train_test_split

X_data, X_test, y_data, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_val shape  : ', X_val.shape)
print('y_val shape  : ', y_val.shape)
print('X_test shape : ', X_test.shape)
print('y_test shape : ', y_test.shape)

## Data Augmentation

Data augmentation is applied to the training and validation data. With the help of data augmentation, the model is fed with random images with random differences. That way, the model is trained with more data. 

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(horizontal_flip=True,
                               rotation_range = 45,
                               zoom_range=0.2,
                               height_shift_range = 0.5,
                               width_shift_range = 0.5)

validation_gen = ImageDataGenerator(horizontal_flip=True,
                               rotation_range = 45,
                               zoom_range=0.2,
                               height_shift_range = 0.5,
                               width_shift_range = 0.5)

train_gen.fit(X_train)
validation_gen.fit(X_val)

## Building Model with Xception

Instead of building a custom CNN model, pre-trained Xception model is used as our base model. 

Global average pooling method is chosen before the classifier dense layer.  

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.optimizers import Adam

# Defining batch and epoch sizes
batch_size = 100
epochs = 15

# Defining the pretrained base model
base = Xception(include_top=False, weights='imagenet', input_shape=(128,128,3))
x = base.output
x = GlobalAveragePooling2D()(x)
# Defining the head of the model where the prediction is conducted
head = Dense(5, activation='softmax')(x)
# Combining base and head 
model = Model(inputs=base.input, outputs=head)

# Compiling the model
model.compile(optimizer=Adam(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics=['accuracy'])

# Fitting the model with train and validation augmented datasets.
history = model.fit_generator(train_gen.flow(X_train, y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = validation_gen.flow(X_val, y_val, batch_size=batch_size),
                              steps_per_epoch = X_train.shape[0] // batch_size)

In [ ]:
history_df = pd.DataFrame(history.history)

plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history_df['loss'], label='training loss')
plt.plot(history_df['val_loss'], label='validation loss')
plt.title('Model Loss Function')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history_df['accuracy'], label='training accuracy')
plt.plot(history_df['val_accuracy'], label='validation accuracy')
plt.title('Model Accuracy')
plt.legend();

## Predicting Test Dataset

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Predicting labels from X_test data
y_pred = model.predict(X_test)

# Converting prediction classes from one hot encoding to list
# Argmax returns the position of the largest value
y_pred_classes = np.argmax(y_pred, axis = 1)

# Convert test labels from one hot encoding to list
y_test_classes = np.argmax(y_test, axis = 1)

# Create the confusion matrix
confmx = confusion_matrix(y_test_classes, y_pred_classes)
f, ax = plt.subplots(figsize = (8,8))
sns.heatmap(confmx, annot=True, fmt='.1f', ax = ax)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show();

In [ ]:
# Printing the model scores:
print(classification_report(y_test_classes, y_pred_classes))

We can see model scores for each categories. The model's performance is good enough with 0.91 accuracy and F1 score.

Now let's use the model with 5 random images.

## Trying the Model with Some Images

In [ ]:
test_img = pd.read_csv('../input/game-of-deep-learning-ship-datasets/test_ApKoW4T.csv')
sample_img = test_img.sample(5)

# List of image names
img_list = list(sample_img['image'])

sample_img_data = []

for each in img_list:
    # Each images' path
    each_path = os.path.join(path, each)
    # Read each image
    each_img = cv2.imread(each_path)
    # OpenCv default color is BGR. Convert it to RGB
    each_img = cv2.cvtColor(each_img, cv2.COLOR_BGR2RGB)
    # Resize the images
    each_img_resized = cv2.resize(each_img, (128,128))
    # Save arrays to a list
    sample_img_data.append(each_img_resized)

# Converting list to numpy array
sample_img_data = np.array(sample_img_data)
print('Shape of X: ', sample_img_data.shape)

In [ ]:
# Predicting random 5 images
sample_pred = model.predict(sample_img_data)
sample_classes = np.argmax(sample_pred, axis = 1)

# Visualizing the predictions
i = 0
plt.figure(figsize=(18,9))
for each in range(4):
    i += 1
    plt.subplot(2,4,i)
    plt.imshow(sample_img_data[each])
    plt.xlabel('PREDICTION: ' + str(categories[sample_classes[each]]))
    plt.xticks([])
    plt.yticks([])

I hope you liked this study and learned something new. Feel free to add your opinions, ask questions in the comment section.

Melih